In [1]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import time
import numpy as np
import matplotlib.pyplot as plt
import math
from deep_audio import Directory, JSON

In [2]:
method_algo = 'mfcc'
model_algo = 'lstm'
n_rate = 24000

DATASET_PATH = f'processed/{method_algo}/{method_algo}_{n_rate}.json'

inputs, targets, mapping = Directory.load_json_data(DATASET_PATH, inputs_fieldname=method_algo)

In [3]:
def create_dense_model():
    # build the network architecture
    model = keras.Sequential([
        # 1st hidden layer
        keras.layers.LSTM(512, input_shape=[inputs.shape[1], inputs.shape[2]], return_sequences=True),
        keras.layers.LSTM(256),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.3),
        # output layer
        keras.layers.Dense(len(mapping), activation='softmax'),
    ])

    return model

In [4]:
# SPLIT DOS DADOS
random_state = 42
# for random_state in [5438, 53, 14]:
#     for _ in range(4):
# split data into train and test set
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs,
                                                                          targets,
                                                                          test_size=0.2,
                                                                          stratify=targets,
                                                                          random_state=random_state)

inputs_train, inputs_valid, targets_train, targets_valid = train_test_split(inputs_train,
                                                                            targets_train,
                                                                            test_size=0.2,
                                                                            stratify=targets_train,
                                                                            random_state=random_state)


In [5]:
# SALVA UMA CÓPIA DOS DADOS DE TESTE

# data = {
#     "mapping": mapping,
#     "labels": targets_test,
#     "mfcc": inputs_test,
# }

# JSON.create_json_file(f'datatest/deep/datatest_{random_state}_{inputs.shape[0]}.json', data)

In [6]:
# CRIA O MODELO
model = create_dense_model()

# COMPILA A REDE
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# SALVA A ESTRUTURA DO MODELO

timestamp = int(time.time())

Directory.create_directory(f'models/{model_algo}/{method_algo}/{timestamp}')

JSON.create_json_file(f'models/{model_algo}/{method_algo}/{timestamp}/model_structure.json', model.to_json())

model_save_filename = f'models/{model_algo}/{method_algo}/{timestamp}/model_weight.h5'

# DECIDE QUANDO PARAR
earlystopping_cb = keras.callbacks.EarlyStopping(patience=300, restore_best_weights=True)

# SALVA OS PESOS
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [ ]:
# TREINA O MODELO
history = model.fit(inputs_train, targets_train,
                    validation_data=(inputs_valid, targets_valid),
                    epochs=10000,
                    batch_size=128,
                    callbacks=[earlystopping_cb, mdlcheckpoint_cb])

Epoch 1/10000
89/89 [==============================] - 87s 923ms/step - loss: 4.6973 - accuracy: 0.0110 - val_loss: 4.6785 - val_accuracy: 0.0184
Epoch 2/10000
89/89 [==============================] - 80s 900ms/step - loss: 4.6158 - accuracy: 0.0320 - val_loss: 4.6183 - val_accuracy: 0.0322
Epoch 3/10000
89/89 [==============================] - 81s 906ms/step - loss: 4.4920 - accuracy: 0.0520 - val_loss: 4.4411 - val_accuracy: 0.0333
Epoch 4/10000
89/89 [==============================] - 80s 895ms/step - loss: 4.2930 - accuracy: 0.0504 - val_loss: 4.2115 - val_accuracy: 0.0524
Epoch 5/10000
89/89 [==============================] - 80s 904ms/step - loss: 4.0338 - accuracy: 0.0751 - val_loss: 3.9647 - val_accuracy: 0.0707
Epoch 6/10000
89/89 [==============================] - 80s 901ms/step - loss: 3.7483 - accuracy: 0.1076 - val_loss: 3.7194 - val_accuracy: 0.1111
Epoch 7/10000
89/89 [==============================] - 80s 902ms/step - loss: 3.4443 - accuracy: 0.1548 - val_loss: 3.4948 -

Epoch 57/10000
89/89 [==============================] - 80s 895ms/step - loss: 0.1210 - accuracy: 0.9745 - val_loss: 2.3298 - val_accuracy: 0.4687
Epoch 58/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.1237 - accuracy: 0.9713 - val_loss: 2.3105 - val_accuracy: 0.4634
Epoch 59/10000
89/89 [==============================] - 80s 898ms/step - loss: 0.1236 - accuracy: 0.9734 - val_loss: 2.2467 - val_accuracy: 0.4715
Epoch 60/10000
89/89 [==============================] - 80s 901ms/step - loss: 0.1217 - accuracy: 0.9724 - val_loss: 2.2762 - val_accuracy: 0.4683
Epoch 61/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.1036 - accuracy: 0.9803 - val_loss: 2.2494 - val_accuracy: 0.4867
Epoch 62/10000
89/89 [==============================] - 80s 899ms/step - loss: 0.0977 - accuracy: 0.9817 - val_loss: 2.2530 - val_accuracy: 0.4797
Epoch 63/10000
89/89 [==============================] - 79s 894ms/step - loss: 0.0911 - accuracy: 0.9827 - val_loss: 2

89/89 [==============================] - 79s 891ms/step - loss: 0.0368 - accuracy: 0.9924 - val_loss: 2.1439 - val_accuracy: 0.5490
Epoch 113/10000
89/89 [==============================] - 80s 902ms/step - loss: 0.0411 - accuracy: 0.9910 - val_loss: 2.2122 - val_accuracy: 0.5363
Epoch 114/10000
89/89 [==============================] - 80s 898ms/step - loss: 0.0428 - accuracy: 0.9880 - val_loss: 2.1718 - val_accuracy: 0.5525
Epoch 115/10000
89/89 [==============================] - 79s 889ms/step - loss: 0.0346 - accuracy: 0.9935 - val_loss: 2.1769 - val_accuracy: 0.5536
Epoch 116/10000
89/89 [==============================] - 80s 897ms/step - loss: 0.0332 - accuracy: 0.9932 - val_loss: 2.1617 - val_accuracy: 0.5585
Epoch 117/10000
89/89 [==============================] - 80s 900ms/step - loss: 0.0371 - accuracy: 0.9917 - val_loss: 2.1399 - val_accuracy: 0.5617
Epoch 118/10000
89/89 [==============================] - 79s 894ms/step - loss: 0.0396 - accuracy: 0.9905 - val_loss: 2.1090 - v

89/89 [==============================] - 43s 480ms/step - loss: 0.0331 - accuracy: 0.9919 - val_loss: 1.9897 - val_accuracy: 0.6013
Epoch 168/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0370 - accuracy: 0.9900 - val_loss: 1.9775 - val_accuracy: 0.6038
Epoch 169/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0300 - accuracy: 0.9937 - val_loss: 1.9945 - val_accuracy: 0.6042
Epoch 170/10000
89/89 [==============================] - 42s 478ms/step - loss: 0.0309 - accuracy: 0.9920 - val_loss: 1.9418 - val_accuracy: 0.6098
Epoch 171/10000
89/89 [==============================] - 42s 478ms/step - loss: 0.0255 - accuracy: 0.9929 - val_loss: 1.9166 - val_accuracy: 0.6116
Epoch 172/10000
89/89 [==============================] - 43s 482ms/step - loss: 0.0227 - accuracy: 0.9952 - val_loss: 1.9495 - val_accuracy: 0.6176
Epoch 173/10000
89/89 [==============================] - 43s 480ms/step - loss: 0.0216 - accuracy: 0.9949 - val_loss: 1.9200 - v

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



89/89 [==============================] - 42s 477ms/step - loss: 0.0122 - accuracy: 0.9977 - val_loss: 1.7582 - val_accuracy: 0.6611
Epoch 243/10000
89/89 [==============================] - 43s 486ms/step - loss: 0.0109 - accuracy: 0.9980 - val_loss: 1.8171 - val_accuracy: 0.6576
Epoch 244/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0138 - accuracy: 0.9964 - val_loss: 1.7488 - val_accuracy: 0.6643
Epoch 245/10000
89/89 [==============================] - 42s 476ms/step - loss: 0.0098 - accuracy: 0.9984 - val_loss: 1.8167 - val_accuracy: 0.6569
Epoch 246/10000
89/89 [==============================] - 43s 478ms/step - loss: 0.0075 - accuracy: 0.9990 - val_loss: 1.7642 - val_accuracy: 0.6682
Epoch 247/10000
89/89 [==============================] - 43s 479ms/step - loss: 0.0128 - accuracy: 0.9965 - val_loss: 1.8199 - val_accuracy: 0.6622
Epoch 248/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0110 - accuracy: 0.9978 - val_loss: 1.8405 - v

89/89 [==============================] - 43s 478ms/step - loss: 0.0135 - accuracy: 0.9970 - val_loss: 1.7076 - val_accuracy: 0.6859
Epoch 298/10000
89/89 [==============================] - 43s 482ms/step - loss: 0.0142 - accuracy: 0.9960 - val_loss: 1.7027 - val_accuracy: 0.6816
Epoch 299/10000
89/89 [==============================] - 43s 477ms/step - loss: 0.0096 - accuracy: 0.9981 - val_loss: 1.6810 - val_accuracy: 0.6806
Epoch 300/10000
89/89 [==============================] - 43s 478ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 1.6715 - val_accuracy: 0.6880
Epoch 301/10000
89/89 [==============================] - 42s 477ms/step - loss: 0.0078 - accuracy: 0.9983 - val_loss: 1.6819 - val_accuracy: 0.6869
Epoch 302/10000
89/89 [==============================] - 42s 475ms/step - loss: 0.0076 - accuracy: 0.9979 - val_loss: 1.7061 - val_accuracy: 0.6852
Epoch 303/10000
89/89 [==============================] - 42s 476ms/step - loss: 0.0082 - accuracy: 0.9978 - val_loss: 1.6665 - v

In [ ]:
# GERA O GRAFICO DE ACURÁCIA
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_accuracy.png')
plt.close()

# GERA O GRÁFICO DE PERCA
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(f'models/{model_algo}/{method_algo}/{timestamp}/graph_loss.png')
plt.close()

In [ ]:
# PEGA A MAIOR E ACURÁCIA
higher_accuracy = model.evaluate(inputs_test, targets_test, batch_size=128)

higher_accuracy = str(int(higher_accuracy[1] * 10000)).zfill(4)

In [ ]:
# RENOMEIA A PASTA
Directory.rename_directory(f'models/{model_algo}/{method_algo}/{timestamp}',
                           f'models/{model_algo}/{method_algo}/acc{higher_accuracy}_seed{random_state}_epochs{len(history.history["accuracy"])}_time{timestamp}')